In [1]:
# Isaac Berez
# 17.01.23
import sys
from scipy.io import mmread
import os
import glob
import pandas as pd
import numpy as np
from pandas_ods_reader import read_ods
from copy import deepcopy
import pprint
import json
import re
from datetime import datetime
import logging
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
from sklearn import preprocessing
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from sklearn.manifold import TSNE
from sklearn import metrics
from sklearn.cluster import DBSCAN
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from collections import Counter
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
import harmonypy as hm
from matplotlib.cm import ScalarMappable
from datetime import date
import mpld3
import hvplot.pandas
import holoviews as hv
from holoviews import opts
import panel as pn
import bokeh
from bokeh.resources import INLINE
from adjustText import adjust_text
from scipy.stats import mannwhitneyu, false_discovery_control, wilcoxon


import dimorph_processing as dp
import cell_comparison as cc
import sex_stats as ss

today = str(date.today())
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
gaba_folder = '/bigdata/isaac/gaba_files/'
#load in all filtered metadata with subclass labels
with open(gaba_folder + 'GABAmeta_data_df_plis_filtered_markers_2024-09-02.json') as json_data:
    gaba_metadata = json.load(json_data)
gaba_metadata = pd.DataFrame.from_dict(gaba_metadata, orient='columns')

In [3]:
len(np.unique(gaba_metadata.loc['cluster_label']))

50

In [24]:
output_folder = '/bigdata/isaac/gaba_files/sex_stats/volcano_plots/092324_sgr_norm_run/'
#output_folder = '/bigdata/isaac/gaba_files/sex_stats/volcano_plots/031024_fix_t_run/'
sig_genes_df = pd.read_csv(output_folder + 'sig_genes_fix_t.csv',header=None)

In [25]:
sig_genes_df.rename(columns = {0:'id',1:'test',2:'gene',3:'delta',4:'-log10(p_adj)'}, 
            inplace = True)

In [26]:
sig_genes_df.head()

,id,test,gene,delta,-log10(p_adj)
0,2,ΔBN_m,0610012G03Rik,-1.359515,1.752553
1,2,ΔBN_m,1700020I14Rik,-1.225389,1.639768
2,2,ΔBN_m,1810037I17Rik,-1.049231,2.014789
3,2,ΔBN_m,2210013O21Rik,-1.424526,1.922974
4,2,ΔBN_m,2210016L21Rik,-1.312858,2.314693


In [83]:
sig_genes_df.loc[:,'-log10(p_adj)']

0         1.752553
1         1.639768
2         2.014789
3         1.922974
4         2.314693
            ...   
278103    0.379409
278104    0.023115
278105    2.197866
278106    0.050231
278107    0.146351
Name: -log10(p_adj), Length: 278108, dtype: float64

In [54]:
sig_genes_df.shape

(278108, 5)

In [59]:
len(list(np.unique(sig_genes_df['gene'])))

10287

In [62]:
sig_genes_df['-log10(p_adj)'].isna().sum()


0

In [10]:
output_folder = '/bigdata/isaac/gaba_files/sex_stats/volcano_plots/092324_sgr_norm_run/'
all_genes_df = pd.read_csv(output_folder + 'all_genes_fix_t.csv',header=None)
all_genes_df.rename(columns = {0:'id',1:'test',2:'gene',3:'delta',4:'-log10(p_adj)'}, 
            inplace = True)
all_genes_df.head()

,id,test,gene,delta,-log10(p_adj)
0,2,ΔBN_m,0610007P14Rik,-0.069888,0.080931
1,2,ΔBN_m,0610009B22Rik,-1.242754,0.564952
2,2,ΔBN_m,0610009L18Rik,-0.512608,0.092727
3,2,ΔBN_m,0610009O20Rik,-0.030448,0.003654
4,2,ΔBN_m,0610010F05Rik,-2.028741,0.809249


In [86]:
np.where(pd.isna(tmp)==True)

(array([    1,     1,     1, ..., 11399, 11399, 11399]),
 array([144, 145, 146, ..., 149, 150, 151]))

In [89]:
output_folder = '/bigdata/isaac/gaba_files/sex_stats/volcano_plots/311024_fix_t_run/'
all_genes_df_fix_t = pd.read_csv(output_folder + 'all_genes_fix_t.csv',header=None)
all_genes_df_fix_t.rename(columns = {0:'id',1:'test',2:'gene',3:'delta',4:'-log10(p_adj)'}, 
            inplace = True)
all_genes_df_fix_t.head()

,id,test,gene,delta,-log10(p_adj)
0,2,ΔBN_m,0610007P14Rik,-0.081082,0.083065
1,2,ΔBN_m,0610009B22Rik,-0.274474,0.590782
2,2,ΔBN_m,0610009L18Rik,-0.064342,0.095316
3,2,ΔBN_m,0610009O20Rik,0.014070,0.003743
4,2,ΔBN_m,0610010F05Rik,-0.344001,0.837539


In [90]:
tmp_fix_t = format_genes(all_genes_df_fix_t,gaba_metadata,output_folder)

In [104]:
tmp_fix_t

,delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),...,delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj)
id,2,2,2,2,2,2,2,2,3,3,...,49,49,50,50,50,50,50,50,50,50
test,ΔBN_f,ΔBN_f,ΔBN_m,ΔBN_m,Δmf_B,Δmf_B,Δmf_N,Δmf_N,ΔBN_f,ΔBN_f,...,Δmf_N,Δmf_N,ΔBN_f,ΔBN_f,ΔBN_m,ΔBN_m,Δmf_B,Δmf_B,Δmf_N,Δmf_N
gene,,,,,,,,,,,,,,,,,,,,,
0610007P14Rik,-0.089003,0.032900,-0.081082,0.083065,-0.042593,0.000802,-0.050513,0.0,-0.017356,0.000000,...,-0.163165,0.022818,-0.489908,0.356479,-0.457375,0.790009,-0.121821,0.0,-0.154355,0.0
0610009B22Rik,-0.311795,0.579071,-0.274474,0.590782,0.094533,0.015932,0.057213,0.0,-0.207602,0.000000,...,-0.035206,0.007134,-0.430928,0.597439,0.273235,0.350164,0.332198,0.0,-0.371966,0.0
0610009L18Rik,0.047458,0.009200,-0.064342,0.095316,-0.093617,0.034226,0.018183,0.0,-0.008062,0.000000,...,-0.091383,0.019677,0.065271,0.002081,-0.094286,0.074926,-0.052414,0.0,0.107143,0.0
0610009O20Rik,0.126641,0.073339,0.014070,0.003743,-0.053795,0.009826,0.058775,0.0,0.096425,0.000000,...,-0.003965,0.000000,0.099754,0.002081,0.004286,0.000000,-0.166897,0.0,-0.071429,0.0
0610010F05Rik,0.005795,0.000000,-0.344001,0.837539,-0.227545,0.165151,0.122250,0.0,0.028714,0.000000,...,-0.229080,0.053541,-0.033927,0.000751,-0.351596,0.408478,-0.061600,0.0,0.256069,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mt-Nd3,0.898420,2.869352,0.272346,0.461215,-0.113740,0.013038,0.512334,0.0,0.932660,3.618014,...,0.498165,0.371154,0.089740,0.000000,0.162625,0.059637,0.045672,0.0,-0.027213,0.0


In [114]:
type(tmp_fix_t.columns)

pandas.core.indexes.multi.MultiIndex

In [129]:
tmp_fix_t.columns.get_level_values(1)

Index([ 2,  2,  2,  2,  2,  2,  2,  2,  3,  3,
       ...
       49, 49, 50, 50, 50, 50, 50, 50, 50, 50],
      dtype='int64', name='id', length=392)

In [96]:
tmp_fix_t.to_excel(output_folder + 'all_genes_formatted.xlsx')

In [91]:
np.where(pd.isna(tmp_fix_t)==True)

(array([], dtype=int64), array([], dtype=int64))

In [101]:
#rename ids to marker names

#load marker dict
folder = '/bigdata/isaac/gaba_files/'
with open (folder + 'GABA_mg_cl_dict_final_2024-09-02.json') as json_data:
    GABA_mg_cl_dict_final = json.load(json_data)

In [122]:
GABA_mg_cl_dict_final = {int(k):v for k,v in GABA_mg_cl_dict_final.items()}

In [124]:
tmp_fix_t_w_ct = tmp_fix_t.copy()

In [144]:
len(list([GABA_mg_cl_dict_final[x] for x in tmp_fix_t.columns.get_level_values(1)]))

392

In [145]:
len(tmp_fix_t.columns.get_level_values(1))

392

In [147]:
ct = [GABA_mg_cl_dict_final[x] for x in tmp_fix_t.columns.get_level_values(1)]

In [148]:
ct_flat = ['-'.join(x) for x in ct]

In [152]:
new_columns = pd.MultiIndex.from_arrays([ct_flat] + [tmp_fix_t_w_ct.columns.get_level_values(i) for i in range(tmp_fix_t_w_ct.columns.nlevels)])
tmp_fix_t_w_ct.columns = new_columns

In [153]:
tmp_fix_t_w_ct

Sncg-Htr3a                                                  \
                   delta -log10(p_adj)     delta -log10(p_adj)     delta   
id                    2             2         2             2         2    
test               ΔBN_f         ΔBN_f     ΔBN_m         ΔBN_m     Δmf_B   
gene                                                                       
0610007P14Rik  -0.089003      0.032900 -0.081082      0.083065 -0.042593   
0610009B22Rik  -0.311795      0.579071 -0.274474      0.590782  0.094533   
0610009L18Rik   0.047458      0.009200 -0.064342      0.095316 -0.093617   
0610009O20Rik   0.126641      0.073339  0.014070      0.003743 -0.053795   
0610010F05Rik   0.005795      0.000000 -0.344001      0.837539 -0.227545   
...                  ...           ...       ...           ...       ...   
mt-Nd3          0.898420      2.869352  0.272346      0.461215 -0.113740   
mt-Nd4         -0.078778      0.001867 -0.150464      0.123095  0.055137   
mt-Nd4l         0.429451      0.625402  0.404615      0.901350  0.120550   
mt-Nd5          0.003780      0.000000  0.059827      0.067523  0.102414   
mt-Nd6          0.017332      0.000350  0.096368      0.125060 -0.054857   

                                                    Vip-Cbln2                \
              -log10(p_adj)     delta -log10(p_adj)     delta -log10(p_adj)   
id                       2         2             2         3             3    
test                  Δmf_B     Δmf_N         Δmf_N     ΔBN_f         ΔBN_f   
gene                                                                          
0610007P14Rik      0.000802 -0.050513           0.0 -0.017356      0.000000   
0610009B22Rik      0.015932  0.057213           0.0 -0.207602      0.000000   
0610009L18Rik      0.034226  0.018183           0.0 -0.008062      0.000000   
0610009O20Rik      0.009826  0.058775           0.0  0.096425      0.000000   
0610010F05Rik      0.165151  0.122250           0.0  0.028714      0.000000   
...                     ...       ...           ...       ...           ...   
mt-Nd3             0.013038  0.512334           0.0  0.932660      3.618014   
mt-Nd4             0.000496  0.126823           0.0 -0.072158      0.000000   
mt-Nd4l            0.002775  0.145386           0.0  0.336804      0.004305   
mt-Nd5             0.001637  0.046366           0.0  0.192596      0.000000   
mt-Nd6             0.009826 -0.133893           0.0  0.052787      0.000000   

               ... Megf11-Gpc5               BC048546-Dlx1                \
               ...       delta -log10(p_adj)         delta -log10(p_adj)   
id             ...          49            49            50            50   
test           ...       Δmf_N         Δmf_N         ΔBN_f         ΔBN_f   
gene           ...                                                         
0610007P14Rik  ...   -0.163165      0.022818     -0.489908      0.356479   
0610009B22Rik  ...   -0.035206      0.007134     -0.430928      0.597439   
0610009L18Rik  ...   -0.091383      0.019677      0.065271      0.002081   
0610009O20Rik  ...   -0.003965      0.000000      0.099754      0.002081   
0610010F05Rik  ...   -0.229080      0.053541     -0.033927      0.000751   
...            ...         ...           ...           ...           ...   
mt-Nd3         ...    0.498165      0.371154      0.089740      0.000000   
mt-Nd4         ...   -0.066036      0.000682     -0.384554      0.484495   
mt-Nd4l        ...    0.475102      0.372740     -0.359720      0.076548   
mt-Nd5         ...    0.052588      0.007134     -0.258438      0.068258   
mt-Nd6         ...    0.334358      0.201822     -0.228762      0.011147   

                                                                         \
                  delta -log10(p_adj)     delta -log10(p_adj)     delta   
id                   50            50        50            50        50   
test              ΔBN_m         ΔBN_m     Δmf_B         Δmf_B     Δmf_N   
gene           

In [155]:
tmp_fix_t_w_ct.to_excel(output_folder + 'all_genes_formatted_fix_w_ct.xlsx')

In [154]:
#tmp_fix_t_w_ct.columns.set_levels(ct_flat,level=1)

{1: ['Chat', 'Isl1'],
 2: ['Sncg', 'Htr3a'],
 3: ['Vip', 'Cbln2'],
 4: ['Cpne4', 'Pde3a'],
 5: ['Reln', 'Ndnf'],
 6: ['Dab1', 'Nfib'],
 7: ['Kit', 'Pdlim5'],
 8: ['Hcrtr2', 'Id2'],
 9: ['Pthlh', 'Pvalb'],
 10: ['Rbp4', 'Kitl'],
 11: ['Moxd1', 'Vwc2'],
 12: ['Maf', 'Mafb'],
 13: ['Npy', 'Sst'],
 14: ['Spon1', 'Tox'],
 15: ['Gm28884', 'Fam159b'],
 16: ['St18', 'Gm17660'],
 17: ['Calcr', 'Peg10'],
 18: ['Prlr', 'Pde1c'],
 19: ['Cbln1', 'Grp'],
 20: ['Tmem100'],
 21: ['Unc13c', 'Crtac1'],
 22: ['Igfbp6', 'Rprm'],
 23: ['Wfs1', 'Prok2'],
 24: ['Jsrp1', 'Col18a1'],
 25: ['Lpl'],
 26: ['Rmst', 'Mgat4c'],
 27: ['Oprk1', 'Trhde'],
 28: ['Lmo1', 'Chn2'],
 29: ['Gal', 'Asb4'],
 30: ['Igsf1', 'A230065H16Rik'],
 31: ['Foxp2', 'Gpr88'],
 32: ['Npy2r'],
 33: ['Nwd2'],
 34: ['Lypd1'],
 35: ['Hs3st4', 'Car10'],
 36: ['BC039966', 'Ngb'],
 37: ['Col23a1'],
 38: ['Col6a1'],
 39: ['Myh7'],
 40: ['Htr1f', 'Tshz1'],
 41: ['Col11a1', 'Pde11a'],
 42: ['Pax6', 'Npnt'],
 43: ['Calca'],
 44: ['Dsc3', 'Mfge8'],
 4

In [60]:
len(list(np.unique(all_genes_df['gene'])))

11414

In [64]:
all_genes_df['delta'].isna().sum()

0

In [11]:
all_genes_df.shape

(1850918, 5)

In [32]:
np.where(sig_genes_df.loc[:,'test']== 'ΔBN_f')

(array([  686,   687,   688, ..., 40437, 40438, 40439]),)

In [33]:
np.where(sig_genes_df.loc[:,'test']== 'Δmf_B')

(array([  745,   746,   747, ..., 34063, 39900, 39901]),)

In [19]:
sig_genes_df.head()

,id,test,gene,delta,-log10(p_adj)
0,2,ΔBN_m,0610012G03Rik,-1.359515,1.752553
1,2,ΔBN_m,1700020I14Rik,-1.225389,1.639768
2,2,ΔBN_m,1810037I17Rik,-1.049231,2.014789
3,2,ΔBN_m,2210013O21Rik,-1.424526,1.922974
4,2,ΔBN_m,2210016L21Rik,-1.312858,2.314693


In [46]:
np.unique(sig_genes_df.loc[:,'test'])

array(['ΔBN_f', 'ΔBN_m', 'Δmf_B', 'Δmf_N'], dtype=object)

In [36]:
sig_genes_df.loc[:,'gene']

0        0610012G03Rik
1        1700020I14Rik
2        1810037I17Rik
3        2210013O21Rik
4        2210016L21Rik
             ...      
40435             Ppia
40436            Sumo2
40437             Tbca
40438            Tmed2
40439             Tpt1
Name: gene, Length: 40440, dtype: object

In [40]:
len(np.unique(sig_genes_df.loc[:,'id']))

43

In [30]:
def format_genes(genes_df, metadata, output_folder, out_file_name, write_to_file = False):
    columns = [item for item in sorted(np.unique(metadata.loc['cluster_label'])) for _ in range(8)] 
    # Pivot the DataFrame to get delta and -log10(p_adj) in different columns for each gene
    genes_df_pivot_delta = genes_df.pivot_table(index='gene', columns=['id', 'test'], values=['delta'])
    # Pivot the DataFrame to get delta and -log10(p_adj) in different columns for each gene
    genes_df_pivot_p = genes_df.pivot_table(index='gene', columns=['id', 'test'], values=['-log10(p_adj)'])
    # Interleave columns of A and B
    new_columns = [col for pair in zip(genes_df_pivot_delta.columns, genes_df_pivot_p.columns) for col in pair]
    genes_df_pivot_combined = pd.concat([genes_df_pivot_delta, genes_df_pivot_p], axis=1)
    genes_df_pivot_combined = genes_df_pivot_combined[new_columns]

    if write_to_file:
        genes_df_pivot_combined.to_csv(output_folder + out_file_name + '.csv')

    return genes_df_pivot_combined

In [12]:
columns = [item for item in sorted(np.unique(gaba_metadata.loc['cluster_label'])) for _ in range(8)] 

In [15]:
amit_sig_genes_df = pd.DataFrame(index=sig_genes_df.loc[:,'gene'], columns = columns)

In [17]:
output_folder

'/bigdata/isaac/gaba_files/sex_stats/volcano_plots/092324_sgr_norm_run/'

In [18]:
amit_sig_genes_df.to_csv(output_folder + 'amit_sig_genes_df.csv')

In [ ]:
file = open(output_folder + 'amit_sig_genes.csv', 'a+', newline ='')
# writing the data into the file
with file:    
    write = csv.writer(file)
    write.writerows(csv_row)

In [38]:
amit_sig_genes_df = pd.read_csv(output_folder + 'amit_sig_genes_df.csv', header = None)

/tmp/ipykernel_3613376/3325566429.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,2

In [51]:
amit_sig_genes_df

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,400
0,gene,1,1,1,1,1,1,1,1,2,...,49,49,50,50,50,50,50,50,50,50
1,NaN,FC_BN_f,q_BN_f,FC_BN_m,q_BN_m,FC_mf_B,q_mf_B,FC_mf_N,q_mf_N,FC_BN_f,...,FC_mf_N,q_mf_N,FC_BN_f,q_BN_f,FC_BN_m,q_BN_m,FC_mf_B,q_mf_B,FC_mf_N,q_mf_N
2,0610012G03Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1700020I14Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1810037I17Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40437,Ppia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40438,Sumo2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40439,Tbca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40440,Tmed2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
amit_sig_genes_df.loc['0610012G03Rik',]

In [21]:
# Pivot the DataFrame to get delta and -log10(p_adj) in different columns for each gene
sig_genes_df_pivot_delta = sig_genes_df.pivot_table(index='gene', columns=['id', 'test'], values=['delta'])


In [85]:
sig_genes_df_pivot_delta

delta                                                            \
id               2                               3                              
test          ΔBN_f     ΔBN_m Δmf_B Δmf_N     ΔBN_f     ΔBN_m Δmf_B     Δmf_N   
gene                                                                            
0610007P14Rik   NaN       NaN   NaN   NaN       NaN       NaN   NaN       NaN   
0610009B22Rik   NaN       NaN   NaN   NaN       NaN       NaN   NaN       NaN   
0610010F05Rik   NaN       NaN   NaN   NaN       NaN       NaN   NaN       NaN   
0610011F06Rik   NaN       NaN   NaN   NaN       NaN       NaN   NaN       NaN   
0610012G03Rik   NaN -1.359515   NaN   NaN       NaN       NaN   NaN       NaN   
...             ...       ...   ...   ...       ...       ...   ...       ...   
mt-Nd3          NaN       NaN   NaN   NaN  1.713297  1.108662   NaN  1.431675   
mt-Nd4          NaN       NaN   NaN   NaN       NaN       NaN   NaN       NaN   
mt-Nd4l         NaN       NaN   NaN   NaN       NaN  1.466503   NaN       NaN   
mt-Nd5          NaN       NaN   NaN   NaN       NaN  1.699038   NaN       NaN   
mt-Nd6          NaN       NaN   NaN   NaN       NaN       NaN   NaN       NaN   

                                   ...                                        \
id                   5             ...        46                48        49   
test              ΔBN_f     ΔBN_m  ...     ΔBN_f ΔBN_m Δmf_N ΔBN_f     ΔBN_f   
gene                               ...                                         
0610007P14Rik       NaN       NaN  ...       NaN   NaN   NaN   NaN -2.228234   
0610009B22Rik       NaN       NaN  ...       NaN   NaN   NaN   NaN       NaN   
0610010F05Rik -3.138791 -2.744166  ...       NaN   NaN   NaN   NaN -2.815631   
0610011F06Rik       NaN       NaN  ... -2.817801   NaN   NaN   NaN       NaN   
0610012G03Rik       NaN -1.528088  ...       NaN   NaN   NaN   NaN -1.350169   
...                 ...       ...  ...       ...   ...   ...   ...       ...   
mt-Nd3              NaN       NaN  ...       NaN   NaN   NaN   NaN       NaN   
mt-Nd4              NaN       NaN  ...       NaN   NaN   NaN   NaN       NaN   
mt-Nd4l             NaN       NaN  ...       NaN   NaN   NaN   NaN       NaN   
mt-Nd5              NaN       NaN  ...       NaN   NaN   NaN   NaN       NaN   
mt-Nd6              NaN       NaN  ...       NaN   NaN   NaN   NaN       NaN   

                                                 
id                                 50            
test          ΔBN_m Δmf_B Δmf_N ΔBN_f     ΔBN_m  
gene                                             
0610007P14Rik   NaN   NaN   NaN   NaN       NaN  
0610009B22Rik   NaN   NaN   NaN   NaN       NaN  
0610010F05Rik   NaN   NaN   NaN   NaN       NaN  
0610011F06Rik   NaN   NaN   NaN   NaN       NaN  
0610012G03Rik   NaN   NaN   NaN   NaN -2.800377  
...             ...   ...   ...   ...       ...  
mt-Nd3          NaN   NaN   NaN   NaN       NaN  
mt-Nd4          NaN   NaN   NaN   NaN       NaN  
mt-Nd4l         NaN   NaN   NaN   NaN       NaN  
mt-Nd5          NaN   NaN   NaN   NaN       NaN  
mt-Nd6          NaN   NaN   NaN   NaN       NaN  

[5355 rows x 130 columns]

In [83]:
sig_genes_df.loc[sig_genes_df.loc[:,'gene'] == 'mt-Nd3',:]

,id,test,gene,delta,-log10(p_adj)
816,3,ΔBN_m,mt-Nd3,1.108662,4.000912
854,3,ΔBN_f,mt-Nd3,1.713297,3.693696
867,3,Δmf_N,mt-Nd3,1.431675,3.106671
5300,7,ΔBN_f,mt-Nd3,1.141092,inf
6636,10,ΔBN_m,mt-Nd3,1.180202,6.879097
8607,12,ΔBN_m,mt-Nd3,1.182591,8.096910
16318,21,Δmf_B,mt-Nd3,1.290419,20.186325
16482,21,Δmf_N,mt-Nd3,1.017935,17.273268
18375,23,Δmf_B,mt-Nd3,1.201756,1.940194
22637,29,ΔBN_f,mt-Nd3,1.017373,8.102373


In [86]:
# Pivot the DataFrame to get delta and -log10(p_adj) in different columns for each gene
sig_genes_df_pivot_p = sig_genes_df.pivot_table(index='gene', columns=['id', 'test'], values=['-log10(p_adj)'])


In [87]:
sig_genes_df_pivot_p

-log10(p_adj)                                                  \
id                       2                               3                    
test                  ΔBN_f     ΔBN_m Δmf_B Δmf_N     ΔBN_f     ΔBN_m Δmf_B   
gene                                                                          
0610007P14Rik           NaN       NaN   NaN   NaN       NaN       NaN   NaN   
0610009B22Rik           NaN       NaN   NaN   NaN       NaN       NaN   NaN   
0610010F05Rik           NaN       NaN   NaN   NaN       NaN       NaN   NaN   
0610011F06Rik           NaN       NaN   NaN   NaN       NaN       NaN   NaN   
0610012G03Rik           NaN  1.752553   NaN   NaN       NaN       NaN   NaN   
...                     ...       ...   ...   ...       ...       ...   ...   
mt-Nd3                  NaN       NaN   NaN   NaN  3.693696  4.000912   NaN   
mt-Nd4                  NaN       NaN   NaN   NaN       NaN       NaN   NaN   
mt-Nd4l                 NaN       NaN   NaN   NaN       NaN  2.813966   NaN   
mt-Nd5                  NaN       NaN   NaN   NaN       NaN  2.149631   NaN   
mt-Nd6                  NaN       NaN   NaN   NaN       NaN       NaN   NaN   

                                             ...                              \
id                             5             ...        46                48   
test              Δmf_N     ΔBN_f     ΔBN_m  ...     ΔBN_f ΔBN_m Δmf_N ΔBN_f   
gene                                         ...                               
0610007P14Rik       NaN       NaN       NaN  ...       NaN   NaN   NaN   NaN   
0610009B22Rik       NaN       NaN       NaN  ...       NaN   NaN   NaN   NaN   
0610010F05Rik       NaN  1.825192  1.742827  ...       NaN   NaN   NaN   NaN   
0610011F06Rik       NaN       NaN       NaN  ...  1.371059   NaN   NaN   NaN   
0610012G03Rik       NaN       NaN  2.472366  ...       NaN   NaN   NaN   NaN   
...                 ...       ...       ...  ...       ...   ...   ...   ...   
mt-Nd3         3.106671       NaN       NaN  ...       NaN   NaN   NaN   NaN   
mt-Nd4              NaN       NaN       NaN  ...       NaN   NaN   NaN   NaN   
mt-Nd4l             NaN       NaN       NaN  ...       NaN   NaN   NaN   NaN   
mt-Nd5              NaN       NaN       NaN  ...       NaN   NaN   NaN   NaN   
mt-Nd6              NaN       NaN       NaN  ...       NaN   NaN   NaN   NaN   

                                                           
id                   49                      50            
test              ΔBN_f ΔBN_m Δmf_B Δmf_N ΔBN_f     ΔBN_m  
gene                                                       
0610007P14Rik  1.501341   NaN   NaN   NaN   NaN       NaN  
0610009B22Rik       NaN   NaN   NaN   NaN   NaN       NaN  
0610010F05Rik  2.688549   NaN   NaN   NaN   NaN       NaN  
0610011F06Rik       NaN   NaN   NaN   NaN   NaN       NaN  
0610012G03Rik  3.348080   NaN   NaN   NaN   NaN  1.909368  
...                 ...   ...   ...   ...   ...       ...  
mt-Nd3              NaN   NaN   NaN   NaN   NaN       NaN  
mt-Nd4              NaN   NaN   NaN   NaN   NaN       NaN  
mt-Nd4l             NaN   NaN   NaN   NaN   NaN       NaN  
mt-Nd5              NaN   NaN   NaN   NaN   NaN       NaN  
mt-Nd6              NaN   NaN   NaN   NaN   NaN       NaN  

[5355 rows x 130 columns]

In [90]:
sig_genes_df_pivot_delta.index.equals(sig_genes_df_pivot_p.index), "DataFrames A and B must have matching indexes."

(True, 'DataFrames A and B must have matching indexes.')

In [93]:
# Interleave columns of A and B
new_columns = [col for pair in zip(sig_genes_df_pivot_delta.columns, sig_genes_df_pivot_p.columns) for col in pair]
sig_genes_df_pivot_combined = pd.concat([sig_genes_df_pivot_delta, sig_genes_df_pivot_p], axis=1)
sig_genes_df_pivot_combined = sig_genes_df_pivot_combined[new_columns]


In [94]:
sig_genes_df_pivot_combined

,delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),...,delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj)
id,2,2,2,2,2,2,2,2,3,3,...,49,49,49,49,49,49,50,50,50,50
test,ΔBN_f,ΔBN_f,ΔBN_m,ΔBN_m,Δmf_B,Δmf_B,Δmf_N,Δmf_N,ΔBN_f,ΔBN_f,...,ΔBN_m,ΔBN_m,Δmf_B,Δmf_B,Δmf_N,Δmf_N,ΔBN_f,ΔBN_f,ΔBN_m,ΔBN_m
gene,,,,,,,,,,,,,,,,,,,,,
0610007P14Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610009B22Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610010F05Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610011F06Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610012G03Rik,NaN,NaN,-1.359515,1.752553,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.800377,1.909368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mt-Nd3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.713297,3.693696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
output_folder

'/bigdata/isaac/gaba_files/sex_stats/volcano_plots/092324_sgr_norm_run/'

In [55]:
sig_tmp = format_genes(sig_genes_df,gaba_metadata,output_folder)
sig_tmp

,delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),...,delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj)
id,2,2,2,2,2,2,2,2,3,3,...,49,49,49,49,49,49,50,50,50,50
test,ΔBN_f,ΔBN_f,ΔBN_m,ΔBN_m,Δmf_B,Δmf_B,Δmf_N,Δmf_N,ΔBN_f,ΔBN_f,...,ΔBN_m,ΔBN_m,Δmf_B,Δmf_B,Δmf_N,Δmf_N,ΔBN_f,ΔBN_f,ΔBN_m,ΔBN_m
gene,,,,,,,,,,,,,,,,,,,,,
0610007P14Rik,-0.584540,0.023142,-0.069888,0.080931,-0.005462,0.001869,-0.520114,0.0,-0.178525,0.000410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610009B22Rik,-1.868569,0.553510,-1.242754,0.564952,0.560475,0.011699,-0.065340,0.0,-1.351357,0.016376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610009L18Rik,0.429137,0.005839,-0.512608,0.092727,-0.846534,0.024348,0.095210,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610009O20Rik,1.006708,0.056640,-0.030448,0.003654,-0.514148,0.008551,0.523007,0.0,0.816582,0.000410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610010F05Rik,0.069187,0.000860,-2.028741,0.809249,-1.528136,0.140176,0.569793,0.0,0.315308,0.000410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mt-Nd3,0.990600,2.839313,0.274220,0.438215,-0.133256,0.010984,0.583124,0.0,1.713297,3.693696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
sig_tmp.loc['0610012G03Rik',:]

               id  test 
delta          2   ΔBN_f   -0.501480
-log10(p_adj)  2   ΔBN_f    0.192155
delta          2   ΔBN_m   -1.359515
-log10(p_adj)  2   ΔBN_m    1.752553
delta          2   Δmf_B   -0.963039
                              ...   
-log10(p_adj)  49  Δmf_N         NaN
delta          50  ΔBN_f         NaN
-log10(p_adj)  50  ΔBN_f         NaN
delta          50  ΔBN_m   -2.800377
-log10(p_adj)  50  ΔBN_m    1.909368
Name: 0610012G03Rik, Length: 278, dtype: float64

In [78]:
-np.log10(0.05)

1.3010299956639813

In [76]:
sig_genes_df.iloc[np.where(sig_genes_df['gene']=='0610007P14Rik')]

,id,test,gene,delta,-log10(p_adj)
4957,7,ΔBN_f,0610007P14Rik,-1.563468,1.683031
10237,16,ΔBN_m,0610007P14Rik,-1.219879,2.082873
14923,21,ΔBN_m,0610007P14Rik,-1.400953,1.657764
15873,21,ΔBN_f,0610007P14Rik,-1.792285,2.067814
16483,22,ΔBN_m,0610007P14Rik,-1.686671,2.289714
20011,27,ΔBN_m,0610007P14Rik,-1.664207,2.076952
21115,27,ΔBN_f,0610007P14Rik,-1.525280,2.803808
23303,32,ΔBN_m,0610007P14Rik,-2.938929,2.323380
29201,36,ΔBN_f,0610007P14Rik,-4.232275,2.312072
29332,37,ΔBN_m,0610007P14Rik,-2.884115,2.477620


In [39]:
tmp = format_genes(all_genes_df,gaba_metadata,output_folder)
tmp

,delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),...,delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj),delta,-log10(p_adj)
id,2,2,2,2,2,2,2,2,3,3,...,49,49,50,50,50,50,50,50,50,50
test,ΔBN_f,ΔBN_f,ΔBN_m,ΔBN_m,Δmf_B,Δmf_B,Δmf_N,Δmf_N,ΔBN_f,ΔBN_f,...,Δmf_N,Δmf_N,ΔBN_f,ΔBN_f,ΔBN_m,ΔBN_m,Δmf_B,Δmf_B,Δmf_N,Δmf_N
gene,,,,,,,,,,,,,,,,,,,,,
0610007P14Rik,-0.584540,0.023142,-0.069888,0.080931,-0.005462,0.001869,-0.520114,0.0,-0.178525,0.000410,...,-1.339847,0.002082,-3.066286,0.373986,-3.179068,0.807889,-0.598322,0.002166,-0.485541,0.0
0610009B22Rik,-1.868569,0.553510,-1.242754,0.564952,0.560475,0.011699,-0.065340,0.0,-1.351357,0.016376,...,-0.376797,0.000430,-3.901375,0.614947,2.395942,0.370806,2.809574,0.002166,-3.487742,0.0
0610009L18Rik,0.429137,0.005839,-0.512608,0.092727,-0.846534,0.024348,0.095210,0.0,NaN,NaN,...,-0.817922,0.000430,0.665860,0.007973,-0.961853,0.092434,-0.534698,0.002166,1.093015,0.0
0610009O20Rik,1.006708,0.056640,-0.030448,0.003654,-0.514148,0.008551,0.523007,0.0,0.816582,0.000410,...,0.014612,0.000430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0610010F05Rik,0.069187,0.000860,-2.028741,0.809249,-1.528136,0.140176,0.569793,0.0,0.315308,0.000410,...,-1.066627,0.015445,-0.146905,0.003356,-2.181062,0.427717,-0.573603,0.002166,1.460553,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mt-Nd3,0.990600,2.839313,0.274220,0.438215,-0.133256,0.010984,0.583124,0.0,1.713297,3.693696,...,0.527292,0.320092,0.387455,0.000717,0.178069,0.077217,0.037600,0.002166,0.246986,0.0


In [33]:
tmp.loc['0610012G03Rik',:]

               id  test 
delta          2   ΔBN_f   -0.501480
-log10(p_adj)  2   ΔBN_f    0.192155
delta          2   ΔBN_m   -1.359515
-log10(p_adj)  2   ΔBN_m    1.752553
delta          2   Δmf_B   -0.963039
                              ...   
-log10(p_adj)  49  Δmf_N         NaN
delta          50  ΔBN_f         NaN
-log10(p_adj)  50  ΔBN_f         NaN
delta          50  ΔBN_m   -2.800377
-log10(p_adj)  50  ΔBN_m    1.909368
Name: 0610012G03Rik, Length: 278, dtype: float64

In [32]:
tmp_old = pd.read_csv(output_folder+'sig_genes_formatted.csv')
tmp_old

/tmp/ipykernel_825485/3124948660.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260) have 

,Unnamed: 0,delta,-log10(p_adj),delta.1,-log10(p_adj).1,delta.2,-log10(p_adj).2,delta.3,-log10(p_adj).3,delta.4,...,delta.125,-log10(p_adj).125,delta.126,-log10(p_adj).126,delta.127,-log10(p_adj).127,delta.128,-log10(p_adj).128,delta.129,-log10(p_adj).129
0,id,2,2,2,2,2,2,2,2,3,...,49,49,49,49,49,49,50,50,50,50
1,test,ΔBN_f,ΔBN_f,ΔBN_m,ΔBN_m,Δmf_B,Δmf_B,Δmf_N,Δmf_N,ΔBN_f,...,ΔBN_m,ΔBN_m,Δmf_B,Δmf_B,Δmf_N,Δmf_N,ΔBN_f,ΔBN_f,ΔBN_m,ΔBN_m
2,gene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0610007P14Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0610009B22Rik,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5353,mt-Nd3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.713297,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5354,mt-Nd4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5355,mt-Nd4l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5356,mt-Nd5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
tmp.to_excel(output_folder + 'all_genes_formatted.xlsx')

In [99]:
sig_genes_df_pivot_combined.to_csv(output_folder + 'sig_genes_formatted.csv')

In [97]:
sig_genes_df.loc[sig_genes_df.loc[:,'gene']=='mt-Nd3',:]

,id,test,gene,delta,-log10(p_adj)
816,3,ΔBN_m,mt-Nd3,1.108662,4.000912
854,3,ΔBN_f,mt-Nd3,1.713297,3.693696
867,3,Δmf_N,mt-Nd3,1.431675,3.106671
5300,7,ΔBN_f,mt-Nd3,1.141092,inf
6636,10,ΔBN_m,mt-Nd3,1.180202,6.879097
8607,12,ΔBN_m,mt-Nd3,1.182591,8.096910
16318,21,Δmf_B,mt-Nd3,1.290419,20.186325
16482,21,Δmf_N,mt-Nd3,1.017935,17.273268
18375,23,Δmf_B,mt-Nd3,1.201756,1.940194
22637,29,ΔBN_f,mt-Nd3,1.017373,8.102373


In [88]:
sig_genes_df

,id,test,gene,delta,-log10(p_adj)
0,2,ΔBN_m,0610012G03Rik,-1.359515,1.752553
1,2,ΔBN_m,1700020I14Rik,-1.225389,1.639768
2,2,ΔBN_m,1810037I17Rik,-1.049231,2.014789
3,2,ΔBN_m,2210013O21Rik,-1.424526,1.922974
4,2,ΔBN_m,2210016L21Rik,-1.312858,2.314693
...,...,...,...,...,...
40435,50,ΔBN_f,Ppia,-1.061595,1.669988
40436,50,ΔBN_f,Sumo2,-1.291682,3.039653
40437,50,ΔBN_f,Tbca,-1.716559,1.567411
40438,50,ΔBN_f,Tmed2,-4.023741,1.699981


In [49]:
#ΔBN_m_df.hvplot.scatter(x='delta', y='-log10(p_adj)', by='gene', 
                  #legend='top', height=400, width=400)